# Basic Instrument Pricing with QuantLib

This notebook demonstrates pricing various financial instruments using QuantLib.

In [ ]:
# Import libraries
import QuantLib as ql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(f"QuantLib version: {ql.__version__}")

## Setup: Evaluation Date

All pricing starts by setting the evaluation date:

In [ ]:
# Set global evaluation date
calculation_date = ql.Date(15, 1, 2025)
ql.Settings.instance().evaluationDate = calculation_date

print(f"Evaluation Date: {calculation_date}")

## Example 1: European Equity Option

Price a European call option using Black-Scholes.

In [ ]:
# Option parameters
underlying_price = 100.0
strike_price = 105.0
volatility = 0.20
risk_free_rate = 0.05
dividend_yield = 0.02
maturity_date = ql.Date(15, 6, 2025)

# Create option
payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

# Market data
spot_handle = ql.QuoteHandle(ql.SimpleQuote(underlying_price))
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, ql.Actual365Fixed())
)
dividend_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_yield, ql.Actual365Fixed())
)
flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, ql.NullCalendar(), volatility, ql.Actual365Fixed())
)

# Pricing engine
bs_process = ql.BlackScholesMertonProcess(spot_handle, dividend_ts, flat_ts, flat_vol_ts)
engine = ql.AnalyticEuropeanEngine(bs_process)
european_option.setPricingEngine(engine)

# Results
print("\n=== European Call Option ===")
print(f"Spot Price: ${underlying_price}")
print(f"Strike Price: ${strike_price}")
print(f"Volatility: {volatility*100:.1f}%")
print(f"\nOption Price: ${european_option.NPV():.2f}")
print(f"Delta: {european_option.delta():.4f}")
print(f"Gamma: {european_option.gamma():.6f}")
print(f"Vega: {european_option.vega():.4f}")
print(f"Theta: {european_option.theta():.4f}")
print(f"Rho: {european_option.rho():.4f}")

## Example 2: American Option Comparison

Compare American and European option prices.

In [ ]:
# American option
american_exercise = ql.AmericanExercise(calculation_date, maturity_date)
american_option = ql.VanillaOption(payoff, american_exercise)

# Binomial engine
binomial_engine = ql.BinomialVanillaEngine(bs_process, "crr", 100)
american_option.setPricingEngine(binomial_engine)

# Comparison
euro_price = european_option.NPV()
amer_price = american_option.NPV()
premium = amer_price - euro_price

print("\n=== European vs American Option ===")
print(f"European Price: ${euro_price:.2f}")
print(f"American Price: ${amer_price:.2f}")
print(f"Early Exercise Premium: ${premium:.2f} ({premium/euro_price*100:.2f}%)")

## Example 3: Volatility Surface Analysis

In [ ]:
# Create volatility surface
strikes = np.linspace(80, 120, 9)
vols = np.linspace(0.15, 0.30, 5)

results = []

for vol in vols:
    vol_prices = []
    for strike in strikes:
        # Create option with new strike
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
        option = ql.VanillaOption(payoff, exercise)
        
        # Update volatility
        vol_ts = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(calculation_date, ql.NullCalendar(), vol, ql.Actual365Fixed())
        )
        process = ql.BlackScholesMertonProcess(spot_handle, dividend_ts, flat_ts, vol_ts)
        option.setPricingEngine(ql.AnalyticEuropeanEngine(process))
        
        vol_prices.append(option.NPV())
    
    results.append(vol_prices)

# Plot
plt.figure(figsize=(12, 6))
for i, vol in enumerate(vols):
    plt.plot(strikes, results[i], marker='o', label=f'Vol = {vol*100:.0f}%')

plt.axvline(x=underlying_price, color='red', linestyle='--', label='Spot Price')
plt.xlabel('Strike Price')
plt.ylabel('Option Price ($)')
plt.title('European Call Option Prices vs Strike and Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Example 4: Interest Rate Swap

Price a vanilla interest rate swap.

In [ ]:
# Swap parameters
nominal = 1_000_000
fixed_rate = 0.05
maturity_years = 5

# Dates
effective_date = calculation_date
termination_date = calculation_date + ql.Period(maturity_years, ql.Years)

# Schedules
fixed_schedule = ql.Schedule(
    effective_date, termination_date,
    ql.Period(ql.Annual), ql.TARGET(),
    ql.ModifiedFollowing, ql.ModifiedFollowing,
    ql.DateGeneration.Forward, False
)

float_schedule = ql.Schedule(
    effective_date, termination_date,
    ql.Period(ql.Semiannual), ql.TARGET(),
    ql.ModifiedFollowing, ql.ModifiedFollowing,
    ql.DateGeneration.Forward, False
)

# Create swap
swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, nominal,
    fixed_schedule, fixed_rate, ql.Actual360(),
    float_schedule, ql.Euribor6M(), 0.0, ql.Actual360()
)

print("\n=== Interest Rate Swap ===")
print(f"Notional: ${nominal:,}")
print(f"Fixed Rate: {fixed_rate*100:.2f}%")
print(f"Maturity: {maturity_years} years")
print(f"\nSwap Type: Payer (pay fixed, receive floating)")
print(f"Fixed Leg Payments: {len(fixed_schedule) - 1}")
print(f"Float Leg Payments: {len(float_schedule) - 1}")

## Example 5: Multiple Instruments Summary

Create a summary table of different instruments.

In [ ]:
# Summary of instruments
instruments_data = {
    'Instrument': [
        'European Call',
        'European Put',
        'American Call',
        'American Put'
    ],
    'Category': ['Equity'] * 4,
    'Strike': [105.0] * 4,
    'Spot': [100.0] * 4
}

# Calculate prices
prices = []

for opt_type in [ql.Option.Call, ql.Option.Put]:
    # European
    payoff = ql.PlainVanillaPayoff(opt_type, strike_price)
    euro = ql.VanillaOption(payoff, exercise)
    euro.setPricingEngine(engine)
    prices.append(euro.NPV())
    
    # American
    amer = ql.VanillaOption(payoff, american_exercise)
    amer.setPricingEngine(binomial_engine)
    prices.append(amer.NPV())

instruments_data['Price ($)'] = prices

# Create DataFrame
df = pd.DataFrame(instruments_data)
print("\n=== Instrument Pricing Summary ===")
print(df.to_string(index=False))

## Summary

This notebook demonstrated:

- ✅ European and American option pricing
- ✅ Greeks calculation
- ✅ Volatility surface analysis
- ✅ Interest rate swap setup
- ✅ Multi-instrument comparison

## Next Steps

- Explore [Advanced Valuation](advanced_valuation.ipynb)
- Review [Instrument Documentation](../instruments/index.md)
- Check [API Reference](../api/index.md)